#심층 신경망 

패션 MNIST 데이터 셋을 분류하면서 케라스로 심층 신경망을 만드는 방법을 배워보기 

In [5]:
#패션 MNIST 데이터셋을 불러오기 

from tensorflow import keras 
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [6]:
#이미지 픽셀값 0~255 범위에서 0~1사이로 변환 
#2차원배열을 1차원 배열로 펼침
#train_test_split()함수로 훈련세트, 검증세트로 나눔 

from sklearn.model_selection import train_test_split
train_scaled = train_input/ 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size =0.2, random_state = 42 
)

In [8]:
#은닉층, 출력층을 케라스의 Dense 클래스로 만들기 

#dense1은 은닉층 100개의 뉴런을 가지고 활성화함수 sigmoid, input_shape의 매개변수에서 입력의 크기 (784,)로 지정 
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

In [9]:
#심층 신경망 만들기 

model= keras.Sequential([dense1, dense2])

In [10]:
#층에 대한 유용한 정보를 얻을 수 있음 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
